<a href="https://colab.research.google.com/github/taylan-sen/CIS355_FALL05/blob/main/Algorithmic_Bias_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
ETHICS OF COMPUTING: ALGORITHMIC FAIRNESS LAB
==============================================
Using the UCI Adult Income Dataset

LEARNING OBJECTIVES:
- Understand how AI can perpetuate historical bias
- Explore different definitions of "fairness"
- Experience trade-offs between accuracy and fairness
- Connect to real-world ethical dilemmas

SETUP INSTRUCTIONS FOR GOOGLE COLAB:
1. Go to colab.research.google.com
2. File -> New Notebook
3. Copy and paste this entire code
4. Run each cell in order (Shift+Enter)

Estimated Time: 60-90 minutes
"""

# ============================================================================
# PART 0: INSTALL AND IMPORT LIBRARIES
# ============================================================================
print("📦 Installing required libraries... (this takes 2-3 minutes)")
print("=" * 60)

!pip install fairlearn scikit-learn pandas numpy matplotlib seaborn -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from fairlearn.metrics import MetricFrame, demographic_parity_ratio, equalized_odds_ratio
from fairlearn.reductions import ExponentiatedGradient, DemographicParity, EqualizedOdds
import warnings
warnings.filterwarnings('ignore')

print("\n✅ Setup complete! Let's explore algorithmic fairness.\n")

# ============================================================================
# PART 1: THE SCENARIO
# ============================================================================
print("=" * 60)
print("🎬 THE SCENARIO")
print("=" * 60)
print("""
You are a Data Scientist at a hiring company. Your job is to build an AI
system that predicts whether job applicants will earn high income (>$50K/year).

Your company will use this to screen applicants for high-paying positions.

THE PROBLEM: The training data comes from 1994 Census records. Historical
hiring practices may have been biased. Will your AI learn and perpetuate
that bias?

Let's find out...
""")

# ============================================================================
# PART 2: LOAD AND EXPLORE THE DATA
# ============================================================================
print("\n" + "=" * 60)
print("📊 LOADING THE UCI ADULT INCOME DATASET")
print("=" * 60)

# Load dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
                'marital-status', 'occupation', 'relationship', 'race', 'sex',
                'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

df = pd.read_csv(url, names=column_names, skipinitialspace=True, na_values='?')

# Clean data
df = df.dropna()
df['income'] = df['income'].map({'<=50K': 0, '>50K': 1})

print(f"\n✓ Loaded {len(df):,} records from 1994 U.S. Census data")
print("\n📋 First few rows:")
print(df.head())

print("\n" + "=" * 60)
print("🔍 KEY QUESTION: Is this historical data biased?")
print("=" * 60)

# Analyze by sex
print("\n1️⃣ Income distribution by SEX:")
sex_income = df.groupby(['sex', 'income']).size().unstack(fill_value=0)
sex_income['high_income_rate'] = (sex_income[1] / (sex_income[0] + sex_income[1]) * 100).round(1)
print(sex_income[['high_income_rate']])

print("\n💭 REFLECTION QUESTION:")
print("   In 1994, men had ~3x higher rate of earning >$50K than women.")
print("   Does this reflect:")
print("   (a) Actual qualifications/merit?")
print("   (b) Historical discrimination?")
print("   (c) Systemic barriers (education access, childcare, etc.)?")
print("   (d) All of the above?")

# Analyze by race
print("\n2️⃣ Income distribution by RACE:")
race_income = df.groupby('race')['income'].apply(lambda x: (x == 1).mean() * 100).round(1).sort_values(ascending=False)
print(race_income)

print("\n⚠️  ETHICAL WARNING:")
print("   If we train AI on this data, it will learn these patterns.")
print("   Question: Is that fair? Is it legal? Should we do it anyway?")

input("\n👉 Press ENTER to continue to model training...")

# ============================================================================
# PART 3: PREPARE DATA FOR MACHINE LEARNING
# ============================================================================
print("\n" + "=" * 60)
print("🔧 PREPARING DATA FOR AI TRAINING")
print("=" * 60)

# Select features
features_to_use = ['age', 'education-num', 'hours-per-week', 'capital-gain', 'sex', 'race']
X = df[features_to_use].copy()
y = df['income']

# Convert categorical to numbers
X['sex'] = (X['sex'] == 'Male').astype(int)
X['race'] = (X['race'] == 'White').astype(int)  # Binary for simplicity

# Store sensitive attribute for fairness analysis
sensitive_features = X['sex'].copy()  # 1=Male, 0=Female

# Split data
X_train, X_test, y_train, y_test, sf_train, sf_test = train_test_split(
    X, y, sensitive_features, test_size=0.3, random_state=42
)

print(f"\n✓ Training set: {len(X_train):,} people")
print(f"✓ Test set: {len(X_test):,} people")
print("\n✓ Features being used:")
for col in features_to_use:
    print(f"   • {col}")

# ============================================================================
# PART 4: TRAIN THE "STANDARD" AI (No Fairness Constraints)
# ============================================================================
print("\n" + "=" * 60)
print("🤖 SCENARIO 1: STANDARD AI (Maximum Accuracy)")
print("=" * 60)
print("\nYou train an AI to maximize accuracy. No fairness constraints.")
print("This is what most companies do by default...\n")

# Train model
model_standard = LogisticRegression(max_iter=1000, random_state=42)
model_standard.fit(X_train, y_train)

# Predictions
y_pred_standard = model_standard.predict(X_test)

# Overall accuracy
accuracy_standard = accuracy_score(y_test, y_pred_standard)
print(f"✓ Overall Accuracy: {accuracy_standard:.1%}")

# Fairness metrics
def analyze_fairness(y_true, y_pred, sensitive_feature, model_name):
    print(f"\n📊 FAIRNESS ANALYSIS: {model_name}")
    print("-" * 60)

    # Selection rates by group
    mf = MetricFrame(
        metrics=lambda y_t, y_p: y_p.mean(),
        y_true=y_true,
        y_pred=y_pred,
        sensitive_features=sensitive_feature
    )

    print("\n1️⃣ SELECTION RATE (% predicted as high-income):")
    print(f"   Female: {mf.by_group[0]:.1%}")
    print(f"   Male:   {mf.by_group[1]:.1%}")

    selection_ratio = mf.by_group[0] / mf.by_group[1]
    print(f"\n   📏 Ratio (Female/Male): {selection_ratio:.2f}")
    print(f"      ↳ 1.0 = perfect parity, <0.8 = potential discrimination")

    # True positive rates (qualified people who get hired)
    female_mask = sensitive_feature == 0
    male_mask = sensitive_feature == 1

    female_tpr = ((y_pred[female_mask] == 1) & (y_true[female_mask] == 1)).sum() / (y_true[female_mask] == 1).sum()
    male_tpr = ((y_pred[male_mask] == 1) & (y_true[male_mask] == 1)).sum() / (y_true[male_mask] == 1).sum()

    print("\n2️⃣ TRUE POSITIVE RATE (% of qualified people accepted):")
    print(f"   Female: {female_tpr:.1%}")
    print(f"   Male:   {male_tpr:.1%}")
    print(f"\n   📏 Difference: {abs(female_tpr - male_tpr):.1%}")

    # Confusion matrix by group
    print("\n3️⃣ CONFUSION MATRICES:")
    print("\n   👩 FEMALE GROUP:")
    cm_female = confusion_matrix(y_true[female_mask], y_pred[female_mask])
    print(f"      True Negative: {cm_female[0,0]:>5}  |  False Positive: {cm_female[0,1]:>5}")
    print(f"      False Negative: {cm_female[1,0]:>4}  |  True Positive: {cm_female[1,1]:>6}")

    print("\n   👨 MALE GROUP:")
    cm_male = confusion_matrix(y_true[male_mask], y_pred[male_mask])
    print(f"      True Negative: {cm_male[0,0]:>5}  |  False Positive: {cm_male[0,1]:>5}")
    print(f"      False Negative: {cm_male[1,0]:>4}  |  True Positive: {cm_male[1,1]:>6}")

    return selection_ratio, female_tpr, male_tpr

ratio1, f_tpr1, m_tpr1 = analyze_fairness(y_test.values, y_pred_standard, sf_test.values, "STANDARD MODEL")

print("\n" + "=" * 60)
print("⚖️  ETHICAL ASSESSMENT")
print("=" * 60)
print("""
LEGAL STANDARD (U.S. EEOC "80% Rule"):
If selection rate for protected group < 80% of other group = potential discrimination

YOUR MODEL'S STATUS:""")
if ratio1 < 0.8:
    print(f"   ❌ FAILS 80% rule ({ratio1:.1%} < 80%)")
    print("   ⚠️  Could face discrimination lawsuit")
else:
    print(f"   ✅ Passes 80% rule ({ratio1:.1%} ≥ 80%)")

print("\n💭 DISCUSSION QUESTIONS:")
print("   1. The AI is 'just learning from data' - who is responsible for bias?")
print("   2. Is high accuracy worth potential discrimination?")
print("   3. At what point does deploying this become gross negligence?")

input("\n👉 Press ENTER to see fairness interventions...")

# ============================================================================
# PART 5: TRAIN WITH DEMOGRAPHIC PARITY CONSTRAINT
# ============================================================================
print("\n" + "=" * 60)
print("🤖 SCENARIO 2: DEMOGRAPHIC PARITY (Equal Selection Rates)")
print("=" * 60)
print("\nYou add a fairness constraint: EQUAL selection rates for both groups.")
print("This means the % of males and females predicted as high-income must be equal.\n")

# Train fair model
constraint_dp = DemographicParity()
mitigator_dp = ExponentiatedGradient(model_standard, constraint_dp)
mitigator_dp.fit(X_train, y_train, sensitive_features=sf_train)

# Predictions
y_pred_dp = mitigator_dp.predict(X_test)

# Accuracy
accuracy_dp = accuracy_score(y_test, y_pred_dp)
print(f"✓ Overall Accuracy: {accuracy_dp:.1%}")
print(f"   ↳ Change from standard: {accuracy_dp - accuracy_standard:+.1%}")

ratio2, f_tpr2, m_tpr2 = analyze_fairness(y_test.values, y_pred_dp, sf_test.values, "DEMOGRAPHIC PARITY MODEL")

print("\n" + "=" * 60)
print("⚖️  TRADE-OFF ANALYSIS")
print("=" * 60)
print(f"\n✅ GAINED: Equal selection rates (ratio now ≈ {ratio2:.2f})")
print(f"❌ LOST: Accuracy decreased by {(accuracy_standard - accuracy_dp):.1%}")
print("\n💭 DISCUSSION QUESTIONS:")
print("   1. Is this 'reverse discrimination' against qualified males?")
print("   2. Who bears the cost of this fairness intervention?")
print("   3. Should we sacrifice accuracy for fairness? How much?")

input("\n👉 Press ENTER to see the third approach...")

# ============================================================================
# PART 6: TRAIN WITH EQUALIZED ODDS CONSTRAINT
# ============================================================================
print("\n" + "=" * 60)
print("🤖 SCENARIO 3: EQUALIZED ODDS (Equal Treatment of Qualified)")
print("=" * 60)
print("\nYou try a different fairness constraint: EQUAL OPPORTUNITY.")
print("Qualified candidates should be accepted at equal rates, regardless of group.\n")

# Train fair model
constraint_eo = EqualizedOdds()
mitigator_eo = ExponentiatedGradient(model_standard, constraint_eo)
mitigator_eo.fit(X_train, y_train, sensitive_features=sf_train)

# Predictions
y_pred_eo = mitigator_eo.predict(X_test)

# Accuracy
accuracy_eo = accuracy_score(y_test, y_pred_eo)
print(f"✓ Overall Accuracy: {accuracy_eo:.1%}")
print(f"   ↳ Change from standard: {accuracy_eo - accuracy_standard:+.1%}")

ratio3, f_tpr3, m_tpr3 = analyze_fairness(y_test.values, y_pred_eo, sf_test.values, "EQUALIZED ODDS MODEL")

print("\n" + "=" * 60)
print("⚖️  COMPARISON OF ALL THREE APPROACHES")
print("=" * 60)

comparison_df = pd.DataFrame({
    'Approach': ['Standard (Max Accuracy)', 'Demographic Parity', 'Equalized Odds'],
    'Accuracy': [f"{accuracy_standard:.1%}", f"{accuracy_dp:.1%}", f"{accuracy_eo:.1%}"],
    'Selection Ratio (F/M)': [f"{ratio1:.2f}", f"{ratio2:.2f}", f"{ratio3:.2f}"],
    'Female TPR': [f"{f_tpr1:.1%}", f"{f_tpr2:.1%}", f"{f_tpr3:.1%}"],
    'Male TPR': [f"{m_tpr1:.1%}", f"{m_tpr2:.1%}", f"{m_tpr3:.1%}"],
})

print("\n")
print(comparison_df.to_string(index=False))

print("\n📊 Visualizing the trade-offs...")

# ============================================================================
# PART 7: VISUALIZE THE FAIRNESS-ACCURACY TRADE-OFF
# ============================================================================

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Accuracy comparison
approaches = ['Standard', 'Demographic\nParity', 'Equalized\nOdds']
accuracies = [accuracy_standard, accuracy_dp, accuracy_eo]
colors = ['red', 'orange', 'green']

axes[0].bar(approaches, accuracies, color=colors, alpha=0.7)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_title('Accuracy Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylim([0.7, 0.9])
for i, v in enumerate(accuracies):
    axes[0].text(i, v + 0.01, f'{v:.1%}', ha='center', fontweight='bold')
axes[0].axhline(y=0.8, color='gray', linestyle='--', alpha=0.5)

# Plot 2: Selection rate ratio
ratios = [ratio1, ratio2, ratio3]
axes[1].bar(approaches, ratios, color=colors, alpha=0.7)
axes[1].set_ylabel('Selection Ratio (Female/Male)', fontsize=12)
axes[1].set_title('Fairness: Selection Parity', fontsize=14, fontweight='bold')
axes[1].axhline(y=1.0, color='green', linestyle='--', linewidth=2, label='Perfect Parity')
axes[1].axhline(y=0.8, color='red', linestyle='--', linewidth=2, label='Legal Threshold')
axes[1].legend()
axes[1].set_ylim([0, 1.2])
for i, v in enumerate(ratios):
    axes[1].text(i, v + 0.05, f'{v:.2f}', ha='center', fontweight='bold')

# Plot 3: True positive rate gap
tpr_gaps = [abs(f_tpr1 - m_tpr1), abs(f_tpr2 - m_tpr2), abs(f_tpr3 - m_tpr3)]
axes[2].bar(approaches, tpr_gaps, color=colors, alpha=0.7)
axes[2].set_ylabel('TPR Gap (|Female - Male|)', fontsize=12)
axes[2].set_title('Fairness: Equal Opportunity', fontsize=14, fontweight='bold')
axes[2].set_ylim([0, 0.15])
for i, v in enumerate(tpr_gaps):
    axes[2].text(i, v + 0.005, f'{v:.1%}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('fairness_tradeoffs.png', dpi=150, bbox_inches='tight')
print("✓ Saved visualization as 'fairness_tradeoffs.png'")
plt.show()

# ============================================================================
# PART 8: THE IMPOSSIBLE THEOREM
# ============================================================================
print("\n" + "=" * 60)
print("🚫 THE IMPOSSIBILITY THEOREM")
print("=" * 60)
print("""
MATHEMATICAL FACT: You cannot satisfy all fairness criteria simultaneously.

For example, you CANNOT have both:
  1. Demographic Parity (equal selection rates)
  AND
  2. Equal Opportunity (equal true positive rates)

...unless the base rates are identical between groups (which they never are).

This means: SOMEONE ALWAYS GETS DISADVANTAGED by your choice of fairness metric.
""")

print("\n💭 ETHICAL QUESTION:")
print("   If perfect fairness is mathematically impossible,")
print("   who should decide which definition of 'fair' we use?")
print("\n   Options:")
print("   (a) The company? (conflict of interest)")
print("   (b) The government? (regulation)")
print("   (c) The affected communities? (democratic input)")
print("   (d) AI ethicists? (expert judgment)")

# ============================================================================
# PART 9: REAL-WORLD CONNECTION
# ============================================================================
print("\n" + "=" * 60)
print("🌍 REAL-WORLD CASE STUDIES")
print("=" * 60)
print("""
1. AMAZON'S RESUME SCREENER (2018)
   • Trained on 10 years of hiring data (mostly male engineers)
   • AI learned to penalize resumes with word "women's" (e.g., "women's chess club")
   • Downranked graduates of all-women's colleges
   • Amazon scrapped the project
   • Question: Was this gross negligence?

2. COMPAS RECIDIVISM ALGORITHM (Criminal Justice)
   • Used to predict re-offending risk
   • ProPublica investigation: Black defendants falsely labeled high-risk 2x more
   • Company claimed algorithm was "race-blind"
   • Still widely used despite controversy
   • Question: Is it worse to have biased AI or biased humans?

3. HIREVUE VIDEO INTERVIEW AI (2019-2021)
   • Analyzed facial expressions and speech patterns
   • Criticized for disability discrimination and lack of transparency
   • Illinois banned it in 2020
   • Company pivoted away from facial analysis
   • Question: Should AI be allowed to judge humans on things we don't understand?

4. APPLE CARD CREDIT LIMITS (2019)
   • Viral tweet: Man offered 20x higher limit than his wife (same income/assets)
   • Apple claimed algorithm was "fair" (no explicit gender variable)
   • Investigation revealed indirect discrimination through other features
   • Question: Is it discrimination if the AI didn't explicitly use protected attributes?
""")

# ============================================================================
# PART 10: YOUR DECISION
# ============================================================================
print("\n" + "=" * 60)
print("🎯 YOUR FINAL DECISION")
print("=" * 60)
print("""
You are the CTO. Your board is demanding you deploy the hiring AI next month.

You have three options:
  [1] Standard Model (85% accurate, fails legal fairness test)
  [2] Demographic Parity (78% accurate, equal selection rates)
  [3] Equalized Odds (82% accurate, equal treatment of qualified)

Each has legal, ethical, and business implications.
""")

choice = input("\nWhich model do you deploy? [1/2/3]: ").strip()

consequences = {
    '1': """
    ⚖️  SIX MONTHS LATER:

    Your company is sued for discrimination. The EEOC investigation reveals:
    • Female applicants were rejected at 2.5x the rate of equally qualified males
    • Your data science team had flagged this risk in internal memos
    • You deployed anyway to maximize accuracy

    Legal assessment: GROSS NEGLIGENCE
    • You had actual knowledge of the bias
    • You proceeded with reckless disregard for harm
    • Settlement: $47 million + mandatory audit for 5 years

    Personal consequences: You are fired. Your professional reputation is destroyed.
    The case becomes a Harvard Business School case study on AI ethics failures.
    """,
    '2': """
    ⚖️  SIX MONTHS LATER:

    Mixed results:
    ✅ No discrimination lawsuits
    ✅ Diversity metrics improve by 40%
    ✅ Company praised by advocacy groups

    ❌ Several highly qualified candidates (both male and female) were rejected
       to maintain equal rates
    ❌ Accuracy drop led to 12% more poor hires
    ❌ CFO questions if the cost was worth it
    ❌ Some employees claim "reverse discrimination"

    You keep your job, but face criticism from multiple sides. The fairness vs.
    accuracy debate continues. At least you can sleep at night knowing you tried.
    """,
    '3': """
    ⚖️  SIX MONTHS LATER:

    Reasonable compromise:
    ✅ Qualified candidates from both groups accepted at equal rates
    ✅ Lawsuit risk is manageable
    ✅ Only modest accuracy loss
    ✅ Board accepts this as the "least bad" option

    ⚠️  Selection rates still differ between groups (which draws some criticism)
    ⚠️  Some argue this doesn't address systemic barriers

    You keep your job. The AI system becomes an industry benchmark for
    "responsible AI deployment." You're invited to speak at conferences
    about navigating fairness trade-offs.
    """
}

print(consequences.get(choice, "\n❌ Invalid choice. But in the real world, doing nothing is also a choice..."))

# ============================================================================
# PART 11: DISCUSSION QUESTIONS FOR CLASS
# ============================================================================
print("\n" + "=" * 60)
print("💭 DISCUSSION QUESTIONS FOR YOUR CLASS")
print("=" * 60)
print("""
1. RESPONSIBILITY & NEGLIGENCE
   • At what point does deploying biased AI become gross negligence?
   • Who is liable: the CEO? CTO? Data scientist? The company?
   • Should individuals face criminal charges for AI harms?

2. FAIRNESS DEFINITIONS
   • Which fairness metric matters most? Why?
   • Who should decide: engineers, lawyers, ethicists, affected communities?
   • Can a "technical" solution ever solve a social problem?

3. TRADE-OFFS
   • How much accuracy should we sacrifice for fairness?
   • Who bears the cost of fairness interventions?
   • Is it ethical to reduce overall performance to help one group?

4. SYSTEMIC ISSUES
   • If all historical data is biased, can we ever train fair AI?
   • Should we use AI for decisions we don't fully understand?
   • Does AI make discrimination worse, or just make it visible?

5. POLICY & REGULATION
   • Should hiring algorithms be regulated? How?
   • Should companies be required to audit AI for bias?
   • What about AI used in criminal justice, lending, healthcare?

6. CONNECTION TO OTHER TOPICS
   • How does this relate to the "racist sinks" example?
   • What about the dermatology AI that looked for rulers?
   • How does this connect to the AI 2027 scenario and alignment?
""")

print("\n" + "=" * 60)
print("✅ LAB COMPLETE!")
print("=" * 60)
print("""
You've experienced firsthand the ethical challenges of AI fairness.

KEY TAKEAWAYS:
• AI learns and amplifies bias from historical data
• Perfect fairness is mathematically impossible
• Every choice disadvantages someone
• Technical solutions alone can't solve social problems
• Transparency and accountability matter

NEXT STEPS:
• Discuss your results with classmates
• Research real-world cases (Amazon, COMPAS, etc.)
• Think about: what regulations should exist?
• Consider: would YOU want to work on AI ethics?

Thank you for engaging seriously with these difficult questions. 🙏
""")

# Optional: Save results for later analysis
results_summary = {
    'standard_accuracy': accuracy_standard,
    'dp_accuracy': accuracy_dp,
    'eo_accuracy': accuracy_eo,
    'standard_ratio': ratio1,
    'dp_ratio': ratio2,
    'eo_ratio': ratio3,
    'choice': choice
}

print("\n📁 Results saved to variable 'results_summary'")
print("   You can access individual values like: results_summary['standard_accuracy']")

📦 Installing required libraries... (this takes 2-3 minutes)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 59.1 MB/s eta 0:00:00

✅ Setup complete! Let's explore algorithmic fairness.

🎬 THE SCENARIO

You are a Data Scientist at a hiring company. Your job is to build an AI 
system that predicts whether job applicants will earn high income (>$50K/year).

Your company will use this to screen applicants for high-paying positions.

THE PROBLEM: The training data comes from 1994 Census records. Historical
hiring practices may have been biased. Will your AI learn and perpetuate 
that bias?

Let's find out...


📊 LOADING THE UCI ADULT INCOME DATASET

✓ Loaded 30,162 records from 1994 U.S. Census data

📋 First few rows:
   age         workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors  